<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EA%B3%BC%EC%A0%95/(%EB%85%B8%EC%9D%B4)%EC%A6%9D%EB%B6%84G_%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%95%99%EC%8A%B5_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 1.3 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595739 sha256=37ce66dc8819fdba8a46ed2b066f50f78c33a4aa74e8c45bacb6fca0b7e3bae3
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.1 MB/s 
     |████████████████████████████████| 769 kB 4.1 MB/s 
     |████████████████████████████████| 895 kB 37.6 MB/s 
     |████████████████████████████████| 3.0 MB 49.6 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-sn4iu6x1
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pi

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본		    G_inc_again_so_model.pt
 산업분류자동화    'My Drive'
'Colab Notebooks'   Screenshot_20220401-001411_Chrome.jpg


In [20]:
import pandas as pd
df=pd.read_csv('/mydrive/산업분류자동화/dataset/inc/df_G_inc.csv')

## 데이터 전처리

In [21]:
df['digit_3'].value_counts()

474    36453
478    30566
452    28176
477    27462
471    26189
472    25668
464    18453
463    17873
462    15510
465    14652
468    14574
461    13968
451    13608
475    13031
467    12096
453     9891
473     9312
479     9218
466     8154
476     7188
Name: digit_3, dtype: int64

In [22]:
df.loc[(df['digit_3'] == 451), 'digit_3'] = 0  
df.loc[(df['digit_3'] == 452), 'digit_3'] = 1  
df.loc[(df['digit_3'] == 453), 'digit_3'] = 2  
df.loc[(df['digit_3'] == 461), 'digit_3'] = 3  
df.loc[(df['digit_3'] == 462), 'digit_3'] = 4  
df.loc[(df['digit_3'] == 463), 'digit_3'] = 5  
df.loc[(df['digit_3'] == 464), 'digit_3'] = 6
df.loc[(df['digit_3'] == 465), 'digit_3'] = 7  
df.loc[(df['digit_3'] == 466), 'digit_3'] = 8  
df.loc[(df['digit_3'] == 467), 'digit_3'] = 9  
df.loc[(df['digit_3'] == 468), 'digit_3'] = 10  
df.loc[(df['digit_3'] == 471), 'digit_3'] = 11  
df.loc[(df['digit_3'] == 472), 'digit_3'] = 12  
df.loc[(df['digit_3'] == 473), 'digit_3'] = 13
df.loc[(df['digit_3'] == 474), 'digit_3'] = 14  
df.loc[(df['digit_3'] == 475), 'digit_3'] = 15  
df.loc[(df['digit_3'] == 476), 'digit_3'] = 16  
df.loc[(df['digit_3'] == 477), 'digit_3'] = 17  
df.loc[(df['digit_3'] == 478), 'digit_3'] = 18  
df.loc[(df['digit_3'] == 479), 'digit_3'] = 19

In [23]:
data_list = []
for q, label in zip(df['text'], df['digit_3'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['매장에서 일반인 대상으로 의류소매', '14'], ['매장에서 일반인대상으로 의류소매', '14'], ['사업장에서 일반인대상 의류판매', '14'], ['일반고객에게 신발매장에서 일반인을 대상으로 신발', '14'], ['일반소비자 가정용 직물제품소매점 커튼 방석 침구류등 소매', '14']]


## Train data & test data

In [24]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [25]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(float(i[label_idx])) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [26]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  0.00005

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [27]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [28]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [29]:
data_train[0]

(array([   2, 2658, 5330, 6903, 2822, 6168, 6877, 6897,  517, 5859, 6168,
        2186, 7318,    3,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(14, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 6)

In [30]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기

In [31]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=20,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [32]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [33]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/4951 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 3.08795166015625 train acc 0.03125
epoch 1 batch id 201 loss 2.861955404281616 train acc 0.04415422885572139
epoch 1 batch id 401 loss 2.3297011852264404 train acc 0.12718204488778054
epoch 1 batch id 601 loss 1.5127151012420654 train acc 0.31551580698835274
epoch 1 batch id 801 loss 0.851445198059082 train acc 0.45925015605493136
epoch 1 batch id 1001 loss 0.6425901055335999 train acc 0.5568337912087912
epoch 1 batch id 1201 loss 0.31535106897354126 train acc 0.6232696711074105
epoch 1 batch id 1401 loss 0.3309275805950165 train acc 0.6708154889364739
epoch 1 batch id 1601 loss 0.18701526522636414 train acc 0.7064042004996877
epoch 1 batch id 1801 loss 0.14551065862178802 train acc 0.7337503470294281
epoch 1 batch id 2001 loss 0.2612721920013428 train acc 0.755872063968016
epoch 1 batch id 2201 loss 0.3905662000179291 train acc 0.774016072239891
epoch 1 batch id 2401 loss 0.09088333696126938 train acc 0.7889290920449813
epoch 1 batch id 2601 loss 0.225700750946

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 9 test acc 0.954230943738657


  0%|          | 0/4951 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.10466939210891724 train acc 0.96875
epoch 10 batch id 201 loss 0.24306608736515045 train acc 0.9748134328358209
epoch 10 batch id 401 loss 0.04420890286564827 train acc 0.9756857855361596
epoch 10 batch id 601 loss 0.006749648600816727 train acc 0.9754575707154742
epoch 10 batch id 801 loss 0.004620939027518034 train acc 0.9751482521847691
epoch 10 batch id 1001 loss 0.1390288770198822 train acc 0.9752591158841158
epoch 10 batch id 1201 loss 0.17370188236236572 train acc 0.9752419858451291
epoch 10 batch id 1401 loss 0.011345806531608105 train acc 0.9752966630977873
epoch 10 batch id 1601 loss 0.04414575919508934 train acc 0.9752205652717052
epoch 10 batch id 1801 loss 0.017012538388371468 train acc 0.9751440172126596
epoch 10 batch id 2001 loss 0.055708710104227066 train acc 0.9751764742628686
epoch 10 batch id 2201 loss 0.23432457447052002 train acc 0.9750184575193094
epoch 10 batch id 2401 loss 0.026715675368905067 train acc 0.9749778738025823
epoch 10 bat

  0%|          | 0/551 [00:00<?, ?it/s]

epoch 10 test acc 0.9543727313974592


In [34]:
#학습 모델 저장
torch.save(model, 'G_inc_so_model_노이2.pt')

In [35]:
model_save_name = 'G_inc_so_model_노이2.pt' #드라이브에 저장(잘불러와지는지는 봐야함..... 나는 안불러와져서 컴퓨터에 다운받은거)
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)